## A Hands-on Workshop series in Machine Learning
### Session 5: Primer on Keras
#### Instructor: Aashita Kesarwani

Now that we know a bit more about neural nets, we will learn how to implement these new ideas with Keras. We will continue to use the Keras python library with TensorFlow backend. Keras provides a user-friendly interface to use TensorFlow and build the models quickly. Both Keras and TensorFlow are robust and powerful python libraries commonly used for deep learning.

There are two ways to build models in Keras, sequential and functional. 
* The sequential API allows us to create layer-by-layer models with multiple inputs and outputs, but is limited in its flexibility. 
* The functional API allows us to create models where we can connect any layer to any other layer, rather than only the layers immediately adjacent to it. It is useful for more advanced architectures where sequential API no longer suffices.

In this primer we will be focusing on the sequential API for ease and clarity.

We will work with [Breast Cancer Wisconsin (Diagnostic) Data Set](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data). Let us first import the data into a pandas dataframe `df`.

In [2]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

df = pd.read_csv('breast_cancer_data.csv')
df.drop(["id","Unnamed: 32"], axis=1, inplace=True)
df.head()

We will train a neural network to classify the cancer cells into malignant and benign, given by the column *diagnosis* in the above dataframe. There are 30 features that describe the characteristics of the cell nuclei of breast masses and are useful indicators to predict whether the cancer is malignant ('M') and benign ('B').

Let us check the number of rows and columns in the dataset.

In [2]:
df.shape

(569, 31)

We extract our input features `X` and the target output `y` from the dataframe `df`:

In [3]:
X = df.drop(["diagnosis"], axis=1)
y = df["diagnosis"].replace({'B': 0, 'M': 1})

Now we want to split the data into training and testing sets. This is for model cross-validation, that is once our network is trained using training data, we want to measure its performance on the unseen validation data.

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)

For the purpose of reproducibility, we seed first.

In [2]:
from numpy.random import seed
seed(10)

import tensorflow 

### Constructing a simple network

We import the relevant functions from [Keras](https://keras.io/) that we plan to use in our network.

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras import losses

input_dim = X_train.shape[1] # size of input variables

Using TensorFlow backend.


To build a squential, feedforward multilayer perceptron we need to start by constructing an appropriately formatted neural network. Let us first look at the shapes of the training and validation sets.

In [7]:
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(426, 30) (143, 30) (426,) (143,)


As we worked out above, we will take in 30 input variables and output a single variable. In this primer we will work out a simple example with one input layer, one hidden layer, and one output layer. Because we are taking in 30 inputs and deterimining one output, we want our first layer to have dimension 30 and our last to have dimension 1.

Let us construct a network with a 30-25-1 architecture, meaning that the input layer have thirty nodes, the hidden layers have 25 nodes, and the output layer has a single node for the binary classification - benign or malignant. 

<img src="images/nn1.svg" />

We begin by creating a Sequential model using [Keras Sequential API](https://keras.io/getting-started/sequential-model-guide/).

In [8]:
model = Sequential()

We begin by creating a Sequential model, then using the [`.add()`](https://keras.io/getting-started/sequential-model-guide/) function to add the desired layers. For this example we want each layer to be dense, so each node in one layer is connected to every node in the next.

Some notes:
* We add layers to the model via the [`.add()`](https://keras.io/getting-started/sequential-model-guide/) method. 
* Keras API allows us to define various kinds of layers that are useful for more involved architectures, but for the Multi-layer Perceptrons, in which every node is connected to every other node in the consequents layers, we use [`Dense`](https://keras.io/layers/core/) layers. 
* The very first layer added to the model needs to be given the input shape (`input_shape`) or input dimension (`input_dim`).
* The other two properties we will specify for the Dense layers are
    * Number of units: We will use 25 nodes for the hidden layer
    * Activation function: We will use sigmoid activation
    
Let us add the first fully-connected (dense) layer with the sigmoid activation using [`.add()`](https://keras.io/getting-started/sequential-model-guide/) method with [`Dense`](https://keras.io/layers/core/) layer. This is the hidden layer with 25 nodes (or units).

In [9]:
model.add(Dense(units=25, input_dim=input_dim, activation="sigmoid")) # Hidden layer

Now, we add the output layer. 
* No need to specify the input size, Keras will automatically take it to be 25 - same as the output size of the previous layer. 
* We specify the the number of units for this layer to be 1 - to give use the prediction of whether the tumor is benign or malignant.

In [10]:
model.add(Dense(units=1, activation="sigmoid")) # Output layer

Let us check the structure of the network using the [`.summary()`](https://keras.io/models/about-keras-models/) function.

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25)                775       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 26        
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


This seems to give us the desired structure. Keep in mind that in practice we want to optimize the number of nodes per hidden layer and number of hidden layers, while in this example the layers were chosen for explanatory purposes.

### Training and testing the model 

Now that we have defined our network architecture, we need to compile it first using [`compile`](https://keras.io/models/model/) before running it. There are a few keywords that we need to pass:
* `optimizer`: We will use the stochastic gradient descent menthod `SGD` for the optimizing the cost/loss function, that we studied earlier. We will set the learning rate (step size) to be `lr=0.001`. There are also a few variants of this method that we will learn and use in the next session.
* `loss`: This is nothing but the cost function. We want to build a model for binary classification and hence will use cross-entropy loss, as covered in the logistic classifier in the last session.
* `metrics`: We studied a lot of classification metrics in our last session. Here, we will simply use accuracy.


In [12]:
model.compile(optimizer=SGD(lr=0.001), loss="binary_crossentropy", metrics=["accuracy"])

Finally, we train the model using 10 rounds (epochs) over the entire training set.

In [13]:
model.fit(X_train, y_train, epochs=10, verbose=1); 

Epoch 1/10
426/426 [==============================] - 0s 926us/step - loss: 0.6244 - acc: 0.8991
Epoch 2/10
426/426 [==============================] - 0s 49us/step - loss: 0.6188 - acc: 0.9085
Epoch 3/10
426/426 [==============================] - 0s 46us/step - loss: 0.6163 - acc: 0.9085
Epoch 4/10
426/426 [==============================] - 0s 46us/step - loss: 0.6135 - acc: 0.9061
Epoch 5/10
426/426 [==============================] - 0s 48us/step - loss: 0.6113 - acc: 0.9038
Epoch 6/10
426/426 [==============================] - 0s 48us/step - loss: 0.6096 - acc: 0.9085
Epoch 7/10
426/426 [==============================] - 0s 44us/step - loss: 0.6074 - acc: 0.9061
Epoch 8/10
426/426 [==============================] - 0s 44us/step - loss: 0.6057 - acc: 0.9061
Epoch 9/10
426/426 [==============================] - 0s 44us/step - loss: 0.6038 - acc: 0.9108
Epoch 10/10
426/426 [==============================] - 0s 49us/step - loss: 0.6011 - acc: 0.9155


In [13]:
model.fit(X_train, y_train, epochs=10, verbose=1); 

Epoch 1/10
426/426 [==============================] - 0s 926us/step - loss: 0.6244 - acc: 0.8991
Epoch 2/10
426/426 [==============================] - 0s 49us/step - loss: 0.6188 - acc: 0.9085
Epoch 3/10
426/426 [==============================] - 0s 46us/step - loss: 0.6163 - acc: 0.9085
Epoch 4/10
426/426 [==============================] - 0s 46us/step - loss: 0.6135 - acc: 0.9061
Epoch 5/10
426/426 [==============================] - 0s 48us/step - loss: 0.6113 - acc: 0.9038
Epoch 6/10
426/426 [==============================] - 0s 48us/step - loss: 0.6096 - acc: 0.9085
Epoch 7/10
426/426 [==============================] - 0s 44us/step - loss: 0.6074 - acc: 0.9061
Epoch 8/10
426/426 [==============================] - 0s 44us/step - loss: 0.6057 - acc: 0.9061
Epoch 9/10
426/426 [==============================] - 0s 44us/step - loss: 0.6038 - acc: 0.9108
Epoch 10/10
426/426 [==============================] - 0s 49us/step - loss: 0.6011 - acc: 0.9155


We note that for each epoch the accuracy is generally increasing and the loss is generally decreasing. We now want to see how well it works on our validation set. 

In [14]:
# show the accuracy on the testing set
print("Evaluating on testing set...")
(loss, accuracy) = model.evaluate(X_valid, y_valid, batch_size=5, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


Evaluating on testing set...
143/143 [==============================] - 0s 578us/step
[INFO] loss=0.6041, accuracy: 88.1119%


In [13]:
model.fit(X_train, y_train, epochs=10, verbose=1); 

Epoch 1/10
426/426 [==============================] - 0s 926us/step - loss: 0.6244 - acc: 0.8991
Epoch 2/10
426/426 [==============================] - 0s 49us/step - loss: 0.6188 - acc: 0.9085
Epoch 3/10
426/426 [==============================] - 0s 46us/step - loss: 0.6163 - acc: 0.9085
Epoch 4/10
426/426 [==============================] - 0s 46us/step - loss: 0.6135 - acc: 0.9061
Epoch 5/10
426/426 [==============================] - 0s 48us/step - loss: 0.6113 - acc: 0.9038
Epoch 6/10
426/426 [==============================] - 0s 48us/step - loss: 0.6096 - acc: 0.9085
Epoch 7/10
426/426 [==============================] - 0s 44us/step - loss: 0.6074 - acc: 0.9061
Epoch 8/10
426/426 [==============================] - 0s 44us/step - loss: 0.6057 - acc: 0.9061
Epoch 9/10
426/426 [==============================] - 0s 44us/step - loss: 0.6038 - acc: 0.9108
Epoch 10/10
426/426 [==============================] - 0s 49us/step - loss: 0.6011 - acc: 0.9155


In [14]:
# show the accuracy on the testing set
print("Evaluating on testing set...")
(loss, accuracy) = model.evaluate(X_valid, y_valid, batch_size=5, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


Evaluating on testing set...
143/143 [==============================] - 0s 578us/step
[INFO] loss=0.6041, accuracy: 88.1119%


#### Acknowledgment:
* The dataset used in this project is the [Breast Cancer Wisconsin (Diagnostic) Data Set](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data).